# Use Case Description

In nearly every security incident, misconfiguration is either the root cause or a critical enabler. Today, most config assessments rely on static rule-matching engines that check config files or cloud settings against predefined rules. But these tools lack contextual understanding of how different components interact — they can't tell if a setting is safe for your environment. This use case focuses on an AI model that ingests configuration data across cloud, infra, app, and identity layers — and reasons over them to answer:

## Model used for this use case
Both Instruct Model and Reasoning Model would be suitable for this task. In this example, we used Reasoning Model.

## SetUp

The setup scripts below are essentially the same as those in the [Quickstart (Reasoning Model)](https://github.com/RobustIntelligence/foundation-ai-cookbook/blob/main/1_quickstarts/Preview_Quickstart_reasoning_model.ipynb)

### Notice
- The code below assumes that users have access to the models via Hugging Face. If you are using API access instead, please replace the inference code with the API version provided in the Quickstart guide.
- This model is currently in preview mode and may receive updates. As a result, outputs can vary even when parameters are configured to ensure reproducibility.

In [1]:
import os
import transformers
import torch
from IPython.display import display, Markdown

HF_TOKEN = os.getenv("HF_TOKEN")

def _get_device():
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

DEVICE = _get_device()

In [2]:
MODEL_ID = "" # To be relaced with the final model name

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float32, # this model's tensor_type is float32
    token=HF_TOKEN,
)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
generation_args = {
    "max_new_tokens": 1024,
    "temperature": None,
    "repetition_penalty": 1.2,
    "do_sample": False,
    "use_cache": True,
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.pad_token_id,
}

In [4]:
import re

def inference(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # The old model version didn’t include the <think> token in the chat template.
    think_token = "<think>\n"
    if not inputs.endswith(think_token):
        inputs += think_token
    
    inputs = tokenizer(inputs, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_args,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens = False)

    # extract the thinking part only
    match = re.search(r"<think>(.*?)<\|end_of_text\|>", response, re.DOTALL)
    
    return match.group(1).strip()

## Config Assessment

Here’s a mock configuration for a Linux-based SSH server:

In [5]:
config_text = """
provider "google" {
  project = "my-project"
  region  = "us-central1"
}

resource "google_compute_network" "default" {
  name                    = "default-network"
  auto_create_subnetworks = true
}

resource "google_compute_firewall" "allow_all_inbound" {
  name    = "allow-all-inbound"
  network = google_compute_network.default.name

  direction = "INGRESS"
  priority  = 1000

  allows {
    protocol = "tcp"
    ports    = ["0-65535"]
  }

  allows {
    protocol = "udp"
    ports    = ["0-65535"]
  }

  source_ranges = ["0.0.0.0/0"]

  target_tags = ["web"]
  description = "Allow all inbound traffic from any IP on all ports"
}
"""


In [6]:
def make_prompt(config_text):
   return f"""You are a security auditor reviewing the configuration of a Terraform configuration file for security issues.
    
    Go over the following configuration line-by-line and assess it for:
    1. Security misconfigurations
    2. Weak or deprecated settings
    3. Missing best practices
    
    ## CONFIGURATION
    {config_text}
    
    Respond with:
    - Detected misconfiguration
    - Severity (Low/Medium/High)
    - Recommended fix
    """

In [7]:
response = inference(make_prompt(config_text))
display(Markdown(response))

## Configuration Review

### Line by Line Analysis:

**Line 1:**
```hcl
provider "google" {
  project = "my-project"
  region  = "us-central1"
}
```
* **Security Misconfiguration:** The provider is configured to use a specific project ID ("my-project") which might be visible in your Terraform state files, potentially exposing this information if shared or committed into version control. However, since this is a common practice when using GCP, unless there's an explicit need to hide the project ID, this isn't considered high risk here.
   * **Severity:** Low
   * **Recommended Fix:** If sensitive data like project IDs should not be exposed, consider using environment variables or secrets management tools like Vault.

**Line 5-7:**
```hcl
resource "google_compute_network" "default" {
  name                    = "default-network"
  auto_create_subnetworks = true
}
```
* **Best Practice Violation:** Enabling `auto_create_subnetworks` can lead to unintended resources being created automatically. For example, creating default subnets that could allow unauthorized access if left unmonitored. It's better to manage subnet creation explicitly rather than relying on automatic defaults.
   * **Severity:** Medium
   * **Recommended Fix:** Set `auto_create_subnetworks` to false and create subnetworks as needed manually. This ensures more controlled resource management.

**Lines 9-22:**
```hcl
resource "google_compute_firewall" "allow_all_inbound" {
 ...
  source_ranges = ["0.0.0.0/0"] # Allows ingress from anywhere
 ...
}
```
* **Security Misconfiguration:** Allowing all inbound traffic (`source_ranges = "0.0.0.0/0"`) exposes services to every possible IP address. Unless intentionally open to the world, this rule poses significant security risks.
   * **Severity:** High
   * **Recommended Fix:** Restrict `source_ranges` to only necessary IPs/CIDRs instead of allowing all. Use VPC peering, Cloud VPN, or similar mechanisms if you must accept traffic from external networks but limit it to known good sources. Also, ensure that the service tagged with 'web' is properly secured elsewhere (e.g., behind a load balancer).

**Additional Notes:**

- **Weak or Deprecated Settings:** No deprecated settings found in the provided code snippet. Always check documentation for latest versions and deprecations.
  
- **Missing Best Practices:**
  - Lack of IAM controls: Resource permissions aren’t specified; make sure appropriate roles are assigned via policies.
  - Logging & Monitoring: Ensure proper logging and monitoring are set up to detect anomalies (not shown in config).
  - Network Segmentation: Not evident whether other segments exist; review overall architecture for segmentation.
  - Version Control: Avoid committing credentials or sensitive info into TF files; use secret managers instead.

In summary, while some configurations may require adjustments based on specific needs, the identified issues above represent potential vulnerabilities or deviations from best practices.